# 装饰器

## 基础

回到问题的本质。对于1个函数:

In [1]:
def func(a,b):
    print(func.__name__)

希望多做点什么，但又不改变函数,可以这样，但是每次都新写wrapper，把a,b传到func，挺麻烦的

In [2]:
def wrapper(c,a,b):
    print('before! c = %s' %c)
    func(a,b)
    print('after! c = %s' %c)

一切函数皆对象，对所有func来个通用的wapper,其实更强调的是wrapper加入的功能，而不是func的功能。

In [15]:
def wrapper_better(func):
    print('before! c = %s' %func)
    def func(a,b):
        print('in function! c = %s' %func)
        print(func.__name__)
    return func

def some_func():
    print('some func')
wrapper_better(some_func)(2,3)

before! c = <function some_func at 0x7f588c590f28>
in function! c = <function wrapper_better.<locals>.func at 0x7f588c44e378>
func


但是`wrapper_better(some_func)(2,3)`调用起来麻烦呀? 

In [13]:
@wrapper_better
def another_func(a,b):
    print('yet another_func excuting..')

before! c = <function another_func at 0x7f588c44e2f0>


@的语法糖好了:@的时候相当于就调用了1次

In [20]:
another_func=wrapper_better(some_func)

before! c = <function some_func at 0x7f588c590f28>


In [21]:
another_func(1,2)

in function! c = <function wrapper_better.<locals>.func at 0x7f588c44e510>
func


装饰器其实就是一个闭包，把一个函数当做参数然后返回一个相同的替代版函数
相当于@的时候，就已经执行了1次
```
another_func = wrapper_better(another_func)
```

现在想wrapper任何函数，而不是形如func(a,b)? 下面的报错呢

In [22]:
another_func(1,2,3)

TypeError: func() takes 2 positional arguments but 3 were given

用func(\*args,\*\*kwarg) !!

In [23]:
def varg_wrapper(func):
    def wrapper(*args,**kwarg):
        print('Done '+ func.__name__)
        func(*args)
    return wrapper

@varg_wrapper
def func_1(a):
    print(a)
    
@varg_wrapper
def func_2(a,b):
    print(a,b)
    
func_1(1)
func_2(1,2)

Done func_1
1
Done func_2
1 2


希望装饰器时，也能传入一些参数,就再包一层(注意是3层了!)

In [24]:
def vargs_wrapper(*wargs, **wkargs):
    def external_wrapper(func):
        print('external ')
        def wrapper(*args,**kwarg):
            print('Done '+ func.__name__)
            if wargs:
                print('varg paras from wrapper %s' % wargs)
            if wkargs:
                print('kw paras from wrapper %s' % wkargs)
                
            func(*args)
        return wrapper
    return external_wrapper

@vargs_wrapper("hey",again="hey man")
def func_1(a):
    print(a)
    
func_1(1)

external 
Done func_1
varg paras from wrapper hey
kw paras from wrapper {'again': 'hey man'}
1


看到@app.route 这种装饰器莫慌, 下面是模拟flask李的route装饰器实现方法.

## 其他装饰

### 函数装饰类

上面都是用1个函数装饰另外1个函数，能不能用函数装饰类呢？当然是可以的..

函数装饰的是类，仍然相当于 
```
oldclass = class_wrapper(oldclass)
```

装饰类干什么呢? 比如类初始化前希望引入某些调用,加些打印什么的

In [25]:
def class_wraper(cls):
    def inner(*args,**kwarg):
        print('before create %s object' % cls.__name__)
        return cls(*args,**kwarg)
    return inner

@class_wraper
class Funny(object):
    pass

Funny()

before create Funny object


### 类装饰函数

In [10]:
class WrapClass(object):
    def __init__(self,func):
        self.func = func
    def __call__(self,*args, **kwarg):
            print('yo?')
            self.func(*args,**kwarg)

@WrapClass
def func(a,b,c="c"):
    print(a,b,c)
    pass

# same as func(1,2,c=3) <=> WrapClass(func)(1,2,c=3)
func(1,2,c=3)

yo?
1 2 3


### 类装饰类


这种比较少见，单是实现上是没有问题的,仍然是!
```
OldClass = WrapClass(OldClass)
```

In [26]:
#这样的class和咸鱼...不，函数有什么区别?
class WrapClass(object):
    def __init__(self,cls):
        self.cls = cls
    def __call__(self):
        print('before creation')
        return self.cls

# @WrapClass
class OldClass:
    def __init__(self,a):
        print('init a ! %s'%a)

#到底是初始化，还是当函数调用?
OldClass("oooo")

init a ! oooo


## 名字代替的问题

In [13]:
def log(func):
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper
@log
def now():
    print('2015-3-25')

now的name变成wapper啦

In [14]:
now.__name__

'wrapper'

以下方式保存原名称:

In [15]:
import functools
def log(func):
    @functools.wraps(func)
    def wrapper(*args, **kw):
        print('call %s():' % func.__name__)
        return func(*args, **kw)
    return wrapper
@log
def now():
    print('2015-3-25')

In [16]:
now.__name__

'now'

## 例子1 装饰器里加回调的注册 

In [28]:
def hook(*args,**kwargs):
    def wrapper(func):
        def action(*f_args,**f_kwargs):
            if 'before' in kwargs and 'before_args' in kwargs:
                kwargs['before'](kwargs['before_args'])
                ret = func(*f_args,**f_kwargs)
                kwargs['after'](kwargs['after'])
            return ret
        return action
    return wrapper
                

In [17]:
def hook(**kwargs):
    def wap(func):
            @functools.wraps(func)
            def wrapper(*args, **kw):
                if 'before' in kwargs:
                    kwargs['before']( kwargs['before_args'] )
                ret = func(*args, **kw)
                if 'after' in kwargs:
                    kwargs['after']( kwargs['after_args'] )
                return ret
            return wrapper
    return wap

def cbs(strs):
    print(strs)
    
def now():
    print('2015-3-25')

In [31]:

def cbs(strs):
    print(strs)
@hook(before=cbs,before_args='before',after=cbs,after_args='after')
def now():
    print('2015-3-25')

In [19]:
now()

before
2015-3-25
after


执行的效果是:`now = hook(xxxx)(now)`

## 例子2  模拟c function static variable

In [20]:
def static_vars(**kwargs):
    """
    static vars decorater
    """
    #funciton static variable as C
    def decorate(func):
        for k in kwargs:
            setattr(func, k, kwargs[k])
        return func
    return decorate

In [21]:
@static_vars(idx=0)
def func():
    func.idx +=1
    print(func.idx)

## 例子3 模拟flask route装饰器实现

In [32]:
class app(object):
    # "router_name":callback 的dict，核心就在这
    _callback = {}
    @classmethod
    def route(cls,router='/'):
        def middle_router(func):
            print('Add route_path=%s' % router)
            app._callback[router] = func
            def inner_router(requests):
                print('? called?')
                #传参进来，又要回调 比较难!
#                 app._args = args
#                 app._kwargs = kwarg
                #这里决定了装饰后函数还能调用吗？这个看需求
                func(*args,**kwarg)
                pass
            return inner_router
        return middle_router
    
    @classmethod
    def serve_forever(cls):
        print('in loop')
        
        import time
        print('in loop %s' % app._callback.keys())
        while(True):
            
            time.sleep(1)
#             def parse_url(): return '/a/b/c'
            def parse_url(): return '/v1/image'
            def read_requests(): return 'requests header'
            #http came
            url = parse_url()
            request = read_requests()
            # is it a router?
#             print('inner got url %s' % url)
            if url in app._callback:
                    # Really callback is here, and pass a receiving requests to callbacks
                    app._callback[url](read_requests)
    
    

# app.route应该是类方法，无非就是用类把装饰函数的实现再统一一下
@app.route('/a/b/c')
def on_response(requests):
    print('/a/b/c on_response')
    return {'status':200}

@app.route('/v1/image')
def on_response(requests):
    print('/v1/image/ on_response')
    return {'status':200}

# loop is neccessary 
app.serve_forever()

Add route_path=/a/b/c
Add route_path=/v1/image
in loop
in loop dict_keys(['/a/b/c', '/v1/image'])
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response
/v1/image/ on_response


KeyboardInterrupt: 

## 总结

所有的装饰，记住装饰的目的是改变被装饰对象，并替换到被装饰对象，华丽的替身...

```
func <=> func_wraper(func)
func(1,2,3) <=> func_wraper(1,2,3)
```